**Thought process for possession tracking:**

*Put this whole thing in a for loop that iterates over game_play_number:*

1. Check for jumpball, get team in possession
2. Check for result from possession:
    1. Check for all kinds shots
        1. If made shot, increment possession - mark kind if two or three
        1. If off_reb, poss++, keep possesion
        2. If def_reb, poss++, other team has poss
    2. Check for foul
        1. Check if foul results in free throws
            1. If no free throws, keep poss
            2. If no rebounds after free throws, other team gets poss
    3. Check for turnover / steal (listed differently)
        1. Other team gets poss
    4. Check for timeouts
        1. Should not affect possession


In [4]:
import pandas as pd

def main():
    game = pd.read_csv("championship_play_by_play.csv")

    home_team = game.iloc[0]["home_team_name"]
    away_team = game.iloc[0]["away_team_name"]

    print(f"home team: {home_team}")
    print(f"away team: {away_team}")
    print()

    home_team_poss, away_team_poss = 0, 0
    hp_flag, ap_flag = False, False
    home_team_poss_res = {"two_fgm": 0, "thr_fgm": 0, "fg_miss": 0, "ft": 0, "ft_miss": 0, "tov": 0, "oreb": 0}
    away_team_poss_res = {"two_fgm": 0, "thr_fgm": 0, "fg_miss": 0, "ft": 0, "ft_miss": 0, "tov": 0, "oreb": 0}

    curr_poss = ""

    for play in game["game_play_number"]:
        play = play-1
        play_type = game.iloc[play]["type_text"]
        play_desc = game.iloc[play]["text"]

        # check for jump balls
        if game.iloc[play]["type_id"] == 615:
            res = game.iloc[play]["text"].split("by", 1)
            team = res[1].strip()

            if team == away_team:
                curr_poss = away_team
                ap_flag = True
                #away_team_poss+=1
            elif team == home_team:
                curr_poss = home_team
                hp_flag = True
                #home_team_poss+=1

        # check for shots taken
        if "Shot" in play_type and play_type != "Block Shot":
            # look for made shots first
            if "made" in play_desc:
                # check for threes
                if "Three Point" in play_desc:
                    if hp_flag :
                        home_team_poss_res['thr_fgm'] += 1
                        home_team_poss += 1
                        hp_flag = False; ap_flag = True
                    elif ap_flag:
                        away_team_poss_res['thr_fgm'] += 1
                        away_team_poss += 1
                        ap_flag = False; hp_flag = True

                # take care of two point shots
                else:
                    if hp_flag :
                        home_team_poss_res['two_fgm'] += 1
                        home_team_poss += 1
                        hp_flag = False; ap_flag = True
                    elif ap_flag:
                        away_team_poss_res['two_fgm'] += 1
                        away_team_poss += 1
                        ap_flag = False; hp_flag = True

            elif "missed" in play_desc: 
                # add missed shot to team totals
                if hp_flag :
                    home_team_poss_res['fg_miss'] += 1
                    # hp_flag = False; ap_flag = True
                elif ap_flag:
                    away_team_poss_res['fg_miss'] += 1
                    # ap_flag = False; hp_flag = True

                # look at the next play
                play += 1
                play_type = game.iloc[play]["type_text"]

                # check for Offensive rebound
                if "Off" in play_type:
                    # print("OFF")
                    # if home team missed shot and got off reb, add to stats and add a possession
                    if hp_flag:
                        # print("HOMEE")
                        home_team_poss += 1
                        home_team_poss_res['oreb'] += 1
                        # go to next play
                        pass
                    # same thing for away team 
                    elif ap_flag:
                        # print("AWAYY")
                        away_team_poss += 1
                        away_team_poss_res['oreb'] += 1 
                        # go to next play
                        pass
                # check for Defensive rebound
                elif "Def" in play_type:
                    # if home team missed shot, away team got def reb
                    if hp_flag:
                        # print("HOMEE")
                        home_team_poss += 1
                        # switch possession
                        hp_flag = False; ap_flag = True
                        # go to next possession
                        pass
                    # if away team missed shot, home team got def reb
                    elif ap_flag:
                        # print("AWAYY")
                        away_team_poss += 1
                        # switch possession
                        ap_flag = False; hp_flag = True
                        # go to next possession
                        pass

        if "Turnover" in play_type:
            if hp_flag:
                # print(f"Play number: {play+1} \t {game.iloc[play]['text']}")
                home_team_poss_res['tov'] += 1
                hp_flag = False; ap_flag = True

            elif ap_flag:
                # print(f"Play number: {play+1} \t {game.iloc[play]['text']}")
                away_team_poss_res['tov'] += 1
                ap_flag = False; hp_flag = True

        if "Foul" in play_type:
            pass



    print(f"Number of {home_team} possessions: {home_team_poss} (home team)\n"+
          f"Number of {away_team} possessions: {away_team_poss} (away team)\n")

    home_score = 2*home_team_poss_res['two_fgm'] + 3*home_team_poss_res['thr_fgm'] + home_team_poss_res['ft']
    away_score = 2*away_team_poss_res['two_fgm'] + 3*away_team_poss_res['thr_fgm'] + away_team_poss_res['ft']
    print(f"GAME SCORE: \t {home_team} {home_score} \t {away_score} {away_team}\n")

    print(f"{home_team} stats: {home_team_poss_res}")
    print()
    print(f"{away_team} stats: {away_team_poss_res}")
    
main()



    

home team: UConn
away team: Purdue

Number of UConn possessions: 56 (home team)
Number of Purdue possessions: 53 (away team)

GAME SCORE: 	 UConn 58 	 57 Purdue

UConn stats: {'two_fgm': 20, 'thr_fgm': 6, 'fg_miss': 35, 'ft': 0, 'ft_miss': 0, 'tov': 10, 'oreb': 13}

Purdue stats: {'two_fgm': 27, 'thr_fgm': 1, 'fg_miss': 27, 'ft': 0, 'ft_miss': 0, 'tov': 7, 'oreb': 7}


In [6]:
import pandas as pd

# Constants
JUMP_BALL_ID = 615

def handle_jump_ball(play, home_team, away_team, curr_poss):
    res = play["text"].split("by", 1)
    team = res[1].strip()

    if team == away_team:
        curr_poss = away_team
    elif team == home_team:
        curr_poss = home_team

    return curr_poss

def handle_shot(play, home_team, away_team, home_team_poss_res, away_team_poss_res, home_team_has_possession):
    if "made" in play["text"]:
        if "Three Point" in play["text"]:
            if home_team_has_possession:
                home_team_poss_res['thr_fgm'] += 1
            else:
                away_team_poss_res['thr_fgm'] += 1
        else:
            if home_team_has_possession:
                home_team_poss_res['two_fgm'] += 1
            else:
                away_team_poss_res['two_fgm'] += 1
    elif "missed" in play["text"]:
        if home_team_has_possession:
            home_team_poss_res['fg_miss'] += 1
        else:
            away_team_poss_res['fg_miss'] += 1

        # Look for Offensive rebound or Defensive rebound in the next play
        next_play_type = play["type_text"]
        if "Off" in next_play_type:
            home_team_poss_res['oreb'] += 1 if home_team_has_possession else 0
            away_team_poss_res['oreb'] += 1 if not home_team_has_possession else 0
        elif "Def" in next_play_type:
            home_team_has_possession = not home_team_has_possession

    return home_team_has_possession

def main():
    game = pd.read_csv("championship_play_by_play.csv")

    home_team = game.iloc[0]["home_team_name"]
    away_team = game.iloc[0]["away_team_name"]

    print(f"Home team: {home_team}")
    print(f"Away team: {away_team}")
    print()

    home_team_poss_res = {"two_fgm": 0, "thr_fgm": 0, "fg_miss": 0, "ft": 0, "ft_miss": 0, "tov": 0, "oreb": 0}
    away_team_poss_res = {"two_fgm": 0, "thr_fgm": 0, "fg_miss": 0, "ft": 0, "ft_miss": 0, "tov": 0, "oreb": 0}

    home_team_has_possession = False
    curr_poss = ""

    for _, play in game.iterrows():
        play_type = play["type_text"]
        

        if play["type_id"] == JUMP_BALL_ID:
            curr_poss = handle_jump_ball(play, home_team, away_team, curr_poss)
            home_team_has_possession = (curr_poss == home_team)
        elif "Shot" in play_type and play_type != "Block Shot":
            home_team_has_possession = handle_shot(play, home_team, away_team, home_team_poss_res, away_team_poss_res, home_team_has_possession)
        elif "Turnover" in play_type:
            if home_team_has_possession:
                home_team_poss_res['tov'] += 1
            else:
                away_team_poss_res['tov'] += 1
            home_team_has_possession = not home_team_has_possession

    # Print results
    print(f"Home team stats: {home_team_poss_res}")
    print(f"Away team stats: {away_team_poss_res}")

main()


Home team: UConn
Away team: Purdue

Home team stats: {'two_fgm': 26, 'thr_fgm': 5, 'fg_miss': 32, 'ft': 0, 'ft_miss': 0, 'tov': 9, 'oreb': 0}
Away team stats: {'two_fgm': 21, 'thr_fgm': 2, 'fg_miss': 30, 'ft': 0, 'ft_miss': 0, 'tov': 8, 'oreb': 0}


# Current working:

In [61]:
import pandas as pd

# Constants
JUMP_BALL_ID = 615

def handle_jump_ball(play, home_team, away_team, curr_poss):
    res = play.text.split("by", 1)
    team = res[1].strip()

    if team == away_team:
        curr_poss = away_team
    elif team == home_team:
        curr_poss = home_team

    return curr_poss

def handle_shot(play, home_team, away_team, home_team_poss_res, away_team_poss_res, home_team_has_possession, player_info):
    player_team = player_info.iloc[0]['team_short_display_name']
    
    if "made" in play.text:
        if "Three Point" in play.text:
            if player_team == home_team:
            #if home_team_has_possession:
                home_team_poss_res['thr_fgm'] += 1
                home_team_poss_res['poss'] += 1
            else:
                away_team_poss_res['thr_fgm'] += 1
                away_team_poss_res['poss'] += 1
                
        # two point shots
        else:
            if player_team == home_team:
            #if home_team_has_possession:
                home_team_poss_res['two_fgm'] += 1
                home_team_poss_res['poss'] += 1
            else:
                away_team_poss_res['two_fgm'] += 1
                away_team_poss_res['poss'] += 1
                
    elif "missed" in play.text:
        if player_team == home_team:
        #if home_team_has_possession:
            home_team_poss_res['fg_miss'] += 1
        else:
            away_team_poss_res['fg_miss'] += 1

def handle_rebound(play, home_team, away_team, home_team_poss_res, away_team_poss_res, home_team_has_possession, player_info):    
    athlete_id = play.athlete_id_1  # Access athlete_id_1 from the tuple
    
    if pd.notna(athlete_id):
        player_team = player_info.iloc[0]['team_short_display_name']

        if "Offensive" in play.text:
            if player_team == home_team:
                home_team_poss_res['oreb'] += 1
                home_team_poss_res['poss'] += 1
                home_team_has_possession = True
            elif player_team == away_team:
                away_team_poss_res['oreb'] += 1
                away_team_poss_res['poss'] += 1
                home_team_has_possession = False

        elif "Defensive" in play.text:
            home_team_has_possession = not home_team_has_possession
            
    # weird instances of rebounds in play-by-play dataset        
    else:
        res = play.text.split()
        team = res[0]
        
        if team == home_team:
            home_team_has_possession = True
            if res[1] == "Offensive":
                home_team_poss_res['oreb'] += 1
                home_team_poss_res['poss'] += 1
        elif team == away_team:
            home_team_has_possession = False
            if res[1] == "Offensive":
                away_team_poss_res['oreb'] += 1
                away_team_poss_res['poss'] += 1

    return home_team_has_possession

def handle_turnover(play, home_team, away_team, home_team_poss_res, away_team_poss_res, home_team_has_possession, player_info):
    athlete_id = play.athlete_id_1  # Access athlete_id_1 from the tuple
    
    if pd.notna(athlete_id):
        player_team = player_info.iloc[0]['team_short_display_name']
        if player_team == home_team:
            home_team_poss_res['tov'] += 1
            home_team_poss_res['poss'] += 1
            home_team_has_possession = False
        elif player_team == away_team:
            away_team_poss_res['tov'] += 1
            away_team_poss_res['poss'] += 1
            home_team_has_possession = True
            
    else:
        res = play.text.split()
        team = res[0]
        
        if team == home_team:
            home_team_poss_res['tov'] += 1
            home_team_poss_res['poss'] += 1
            home_team_has_possession = False
        elif team == away_team:
            away_team_poss_res['tov'] += 1
            away_team_poss_res['poss'] += 1
            home_team_has_possession = True
            
    return home_team_has_possession

def handle_foul(play, home_team, away_team, home_team_poss_res, away_team_poss_res, home_team_has_possession, player_info):
    player_team = player_info.iloc[0]['team_short_display_name']
    
    if player_team == home_team:
        away_team_poss_res['poss'] += 1 
        home_team_has_possession = False
    elif player_team == away_team:
        home_team_poss_res['poss'] += 1 
        home_team_has_possession = True
        
    return home_team_has_possession

def handle_free_throw(play, home_team, away_team, home_team_poss_res, away_team_poss_res, player_info):
    player_team = player_info.iloc[0]['team_short_display_name']
    if "made" in play.text:
        if player_team == home_team: 
            home_team_poss_res['ft'] += 1
        elif player_team == away_team:
            away_team_poss_res['ft'] += 1
    elif "miss" in play.text:
        if player_team == home_team: 
            home_team_poss_res['ft_miss'] += 1
        elif player_team == away_team:
            away_team_poss_res['ft_miss'] += 1
    #return

def main():
    players = pd.read_csv("players_table.csv")
    
    game = pd.read_csv("championship_play_by_play.csv")

    home_team = game.iloc[0]["home_team_name"]
    away_team = game.iloc[0]["away_team_name"]

    print(f"Home team: {home_team}")
    print(f"Away team: {away_team}")
    print()

    home_team_poss_res = {"two_fgm": 0, "thr_fgm": 0, "fg_miss": 0, "ft": 0, "ft_miss": 0, "tov": 0, "oreb": 0, "poss": 0}
    away_team_poss_res = {"two_fgm": 0, "thr_fgm": 0, "fg_miss": 0, "ft": 0, "ft_miss": 0, "tov": 0, "oreb": 0, "poss": 0}

    home_team_has_possession = False
    curr_poss = ""

    for current_row, next_row in zip(game.itertuples(), game.iloc[1:].itertuples()):
        play = current_row
        play_type = play.type_text
        
        player_id = play.athlete_id_1
        player_info = players.loc[players['athlete_id'] == player_id]

        # next_play = next_row
        # next_play_type = next_play.type_text

        if play.type_id == JUMP_BALL_ID:
            curr_poss = handle_jump_ball(play, home_team, away_team, curr_poss)
            home_team_has_possession = (curr_poss == home_team)
            
        elif "Shot" in play_type and play_type != "Block Shot":
            handle_shot(play, home_team, away_team, home_team_poss_res, away_team_poss_res, home_team_has_possession, player_info)
            
        elif "Rebound" in play_type:
            home_team_has_possession = handle_rebound(play, home_team, away_team, home_team_poss_res, away_team_poss_res, home_team_has_possession, player_info)
            
        elif "Turnover" in play_type:
            home_team_has_possession = handle_turnover(play, home_team, away_team, home_team_poss_res, away_team_poss_res, home_team_has_possession, player_info)
            
        elif "Foul" in play_type:
            fouler = int(play.athlete_id_1)
            player_info = players.loc[players['athlete_id'] == fouler]
            home_team_has_possession = handle_foul(play, home_team, away_team, home_team_poss_res, away_team_poss_res, home_team_has_possession, player_info)
            
        elif "FreeThrow" in play_type:
            handle_free_throw(play, home_team, away_team, home_team_poss_res, away_team_poss_res, player_info)
            
    home_score = 2*home_team_poss_res['two_fgm'] + 3*home_team_poss_res['thr_fgm'] + home_team_poss_res['ft']
    away_score = 2*away_team_poss_res['two_fgm'] + 3*away_team_poss_res['thr_fgm'] + away_team_poss_res['ft']
    print(f"GAME SCORE: \t {home_team} {home_score} \t {away_score} {away_team}\n")
            

    print(f"Home team stats: {home_team_poss_res}")
    print(f"Away team stats: {away_team_poss_res}")
    
    

main()

Home team: UConn
Away team: Purdue

GAME SCORE: 	 UConn 75 	 60 Purdue

Home team stats: {'two_fgm': 24, 'thr_fgm': 6, 'fg_miss': 32, 'ft': 9, 'ft_miss': 2, 'tov': 8, 'oreb': 14, 'poss': 67}
Away team stats: {'two_fgm': 23, 'thr_fgm': 1, 'fg_miss': 30, 'ft': 11, 'ft_miss': 4, 'tov': 9, 'oreb': 9, 'poss': 60}


# NEW CURRENT ONE:

In [65]:
import pandas as pd

# Constants
JUMP_BALL_ID = 615

# def handle_jump_ball(play, home_team, away_team, curr_poss):
#     res = play.text.split("by", 1)
#     team = res[1].strip()

#     if team == away_team:
#         curr_poss = away_team
#     elif team == home_team:
#         curr_poss = home_team

#     return curr_poss

def handle_shot(play, home_team, away_team, home_team_poss_res, away_team_poss_res, home_team_has_possession, player_info):
    player_team = player_info.iloc[0]['team_short_display_name']
    
    if "made" in play.text:
        if "Three Point" in play.text:
            if player_team == home_team:
            #if home_team_has_possession:
                home_team_poss_res['thr_fgm'] += 1
                home_team_poss_res['poss'] += 1
            else:
                away_team_poss_res['thr_fgm'] += 1
                away_team_poss_res['poss'] += 1
                
        # two point shots
        else:
            if player_team == home_team:
            #if home_team_has_possession:
                home_team_poss_res['two_fgm'] += 1
                home_team_poss_res['poss'] += 1
            else:
                away_team_poss_res['two_fgm'] += 1
                away_team_poss_res['poss'] += 1
                
    elif "missed" in play.text:
        if player_team == home_team:
        #if home_team_has_possession:
            home_team_poss_res['fg_miss'] += 1
        else:
            away_team_poss_res['fg_miss'] += 1

def handle_rebound(play, home_team, away_team, home_team_poss_res, away_team_poss_res, home_team_has_possession, player_info):    
    athlete_id = play.athlete_id_1  # Access athlete_id_1 from the tuple
    
    if pd.notna(athlete_id):
        player_team = player_info.iloc[0]['team_short_display_name']

        if "Offensive" in play.text:
            if player_team == home_team:
                home_team_poss_res['oreb'] += 1
                home_team_poss_res['poss'] += 1
                home_team_has_possession = True
            elif player_team == away_team:
                away_team_poss_res['oreb'] += 1
                away_team_poss_res['poss'] += 1
                home_team_has_possession = False

        elif "Defensive" in play.text:
            home_team_has_possession = not home_team_has_possession
            
    # weird instances of rebounds in play-by-play dataset        
    else:
        res = play.text.split()
        team = res[0]
        
        if team == home_team:
            home_team_has_possession = True
            if res[1] == "Offensive":
                home_team_poss_res['oreb'] += 1
                home_team_poss_res['poss'] += 1
        elif team == away_team:
            home_team_has_possession = False
            if res[1] == "Offensive":
                away_team_poss_res['oreb'] += 1
                away_team_poss_res['poss'] += 1

    #return home_team_has_possession

def handle_turnover(play, home_team, away_team, home_team_poss_res, away_team_poss_res, home_team_has_possession, player_info):
    athlete_id = play.athlete_id_1  # Access athlete_id_1 from the tuple
    
    if pd.notna(athlete_id):
        player_team = player_info.iloc[0]['team_short_display_name']
        if player_team == home_team:
            home_team_poss_res['tov'] += 1
            home_team_poss_res['poss'] += 1
            home_team_has_possession = False
        elif player_team == away_team:
            away_team_poss_res['tov'] += 1
            away_team_poss_res['poss'] += 1
            home_team_has_possession = True
            
    else:
        res = play.text.split()
        team = res[0]
        
        if team == home_team:
            home_team_poss_res['tov'] += 1
            home_team_poss_res['poss'] += 1
            home_team_has_possession = False
        elif team == away_team:
            away_team_poss_res['tov'] += 1
            away_team_poss_res['poss'] += 1
            home_team_has_possession = True
            
    #return home_team_has_possession

def handle_foul(play, home_team, away_team, home_team_poss_res, away_team_poss_res, home_team_has_possession, player_info):
    player_team = player_info.iloc[0]['team_short_display_name']
    
    if player_team == home_team:
        away_team_poss_res['poss'] += 1 
        home_team_has_possession = False
    elif player_team == away_team:
        home_team_poss_res['poss'] += 1 
        home_team_has_possession = True
        
    #return home_team_has_possession

def handle_free_throw(play, home_team, away_team, home_team_poss_res, away_team_poss_res, player_info):
    player_team = player_info.iloc[0]['team_short_display_name']
    if "made" in play.text:
        if player_team == home_team: 
            home_team_poss_res['ft'] += 1
        elif player_team == away_team:
            away_team_poss_res['ft'] += 1
    elif "miss" in play.text:
        if player_team == home_team: 
            home_team_poss_res['ft_miss'] += 1
        elif player_team == away_team:
            away_team_poss_res['ft_miss'] += 1
    #return

def main():
    players = pd.read_csv("players_table.csv")
    
    game = pd.read_csv("championship_play_by_play.csv")

    home_team = game.iloc[0]["home_team_name"]
    away_team = game.iloc[0]["away_team_name"]

    print(f"Home team: {home_team}")
    print(f"Away team: {away_team}")
    print()

    home_team_poss_res = {"two_fgm": 0, "thr_fgm": 0, "fg_miss": 0, "ft": 0, "ft_miss": 0, "tov": 0, "oreb": 0, "poss": 0}
    away_team_poss_res = {"two_fgm": 0, "thr_fgm": 0, "fg_miss": 0, "ft": 0, "ft_miss": 0, "tov": 0, "oreb": 0, "poss": 0}

    home_team_has_possession = False
    curr_poss = ""

    for row in game.itertuples():
        play = row
        play_type = play.type_text
        
        player_id = play.athlete_id_1
        player_info = players.loc[players['athlete_id'] == player_id]
        
            
        if "Shot" in play_type and play_type != "Block Shot":
            handle_shot(play, home_team, away_team, home_team_poss_res, away_team_poss_res, home_team_has_possession, player_info)
            
        elif "Rebound" in play_type:
            handle_rebound(play, home_team, away_team, home_team_poss_res, away_team_poss_res, home_team_has_possession, player_info)
            
        elif "Turnover" in play_type:
            handle_turnover(play, home_team, away_team, home_team_poss_res, away_team_poss_res, home_team_has_possession, player_info)
            
        elif "Foul" in play_type:
            fouler = int(play.athlete_id_1)
            player_info = players.loc[players['athlete_id'] == fouler]
            handle_foul(play, home_team, away_team, home_team_poss_res, away_team_poss_res, home_team_has_possession, player_info)
            
        elif "FreeThrow" in play_type:
            handle_free_throw(play, home_team, away_team, home_team_poss_res, away_team_poss_res, player_info)
            
    home_score = 2*home_team_poss_res['two_fgm'] + 3*home_team_poss_res['thr_fgm'] + home_team_poss_res['ft']
    away_score = 2*away_team_poss_res['two_fgm'] + 3*away_team_poss_res['thr_fgm'] + away_team_poss_res['ft']
    print(f"GAME SCORE: \t {home_team} {home_score} \t {away_score} {away_team}\n")
            

    print(f"Home team stats: {home_team_poss_res}")
    print(f"Away team stats: {away_team_poss_res}")
    
    

main()

Home team: UConn
Away team: Purdue

GAME SCORE: 	 UConn 75 	 60 Purdue

Home team stats: {'two_fgm': 24, 'thr_fgm': 6, 'fg_miss': 32, 'ft': 9, 'ft_miss': 2, 'tov': 8, 'oreb': 14, 'poss': 67}
Away team stats: {'two_fgm': 23, 'thr_fgm': 1, 'fg_miss': 30, 'ft': 11, 'ft_miss': 4, 'tov': 9, 'oreb': 9, 'poss': 60}
